<a href="https://colab.research.google.com/github/Abdulrauf1122/Coronary_artery_stenosis_detection/blob/main/Stenosis_Detection_Faster_RCNN_Inception_ResNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data

# **Cloning my_Repository**

In [ ]:
!git clone https://github.com/Abdulrauf1122/Coronary_artery_stenosis_detection.git

# **Installing necessary libraries**

In [ ]:
import os
%cd /content

!pip uninstall opencv-python opencv-contrib-python imgaug -y
!pip install imgaug==0.2.5 opencv-python-headless==4.1.2.30 lvis

!pip install tf-models-official==2.8 tensorflow-gpu==2.8 keras==2.8 tensorboard==2.8 tensorflow-estimator==2.8
!pip install tensorflow.io

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!apt install --allow-change-held-packages libcudnn8=8.4.0.27-1+cuda11.1

!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

import tensorflow as tf
print("Installed Tensorflow version >>> ", tf.__version__)

# **Downloading the TensorFlow Model Garden**

In [ ]:
!git clone --quiet https://github.com/tensorflow/models.git

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python /content/models/research/object_detection/builders/model_builder_test.py

# **Protobuf Installation/Compilation**

In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

# **COCO API installation**

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cp -r pycocotools /content/models/research

# **Test object detection API**

In [ ]:
%cd /content/models/research
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py

# **Downloading Dataset**

In [ ]:
from tqdm import tqdm
import zipfile
import time

%cd /content/
st = time.time()
q = input("Are you sure? Y/N :")
if q == 'Y':
    !wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/ydrm75xywg-1.zip
    with zipfile.ZipFile('/content/ydrm75xywg-1.zip') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting preliminary'):
            try:
                zf.extract(member, "/content/dataset")
            except zipfile.error as e:
                pass
    !rm /content/ydrm75xywg-1.zip
    with zipfile.ZipFile('/content/dataset/ydrm75xywg-1/Stenosis detection.zip') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting final dataset'):
            try:
                zf.extract(member, "/content/stenosis_dataset")
            except zipfile.error as e:
                pass
    print("Cleaning up after downlaod and extraction")
    !rm -rf /content/dataset
    !rm -rf /content/stenosis_dataset/video_test
    !rm -rf /content/stenosis_dataset/video_val
    print("Download & extraction completed")
else:
    print("Moving on ...")

et = time.time()
elapsed_time = et - st
print('\nExecution time:', elapsed_time/60, ' minutes')

# **Creating folder structure**

In [ ]:
import os
workspace_path = '/content/drive/MyDrive/workspace/'
workspace_model_path = workspace_path + 'stenosis_detection_FRCNN/'
annotation_path = workspace_model_path + 'annotations/'
exported_model_path = workspace_model_path + 'exported-models/'
images_path = workspace_model_path + 'images/'
trained_model_path = workspace_model_path + 'trained_model/'
pretrained_model_path = workspace_model_path + 'pre-trained-models/'

try:
    os.mkdir(workspace_path)
    print("Created workspace")
except FileExistsError :
    print("workspace directory already exists: creating rest of the structure inside workspace")
try:
    os.mkdir(workspace_model_path)
    os.mkdir(annotation_path)
    os.mkdir(exported_model_path)
    os.mkdir(images_path)
    os.mkdir(trained_model_path)
    os.mkdir(pretrained_model_path)
    print("full directory structure creation done")
except FileExistsError :
    print("directory structure already exists: Will be using existing directories")

# **Creating labelmap.pbtxt**

In [ ]:
labels = [{'name':'Stenosis', 'id':1}]
with open(annotation_path + "label_map.pbtxt", 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# **Partitioning the dataset**

In [ ]:
st = time.time()
q = input("Are you sure? Y/N :")
if q == 'Y':
    if os.path.isdir("/content/stenosis_dataset/dataset"):
        !rm -rf /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/images/
        os.mkdir(images_path)

        !python /content/coronary-artery-stenosis-detection/partition_dataset.py -x -i /content/stenosis_dataset/dataset -o /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/images -r 0.1

        print("Dataset split into Train & Test")
        print("Cleaning up after partitioning dataset")
        !rm -rf /content/stenosis_dataset
    else:
      print("Dataset doesn't exists")
else:
    print('Moving on ...')
et = time.time()
elapsed_time = et - st
print('\nExecution time:', elapsed_time/60 , 'minutes')

# **Generating TF records**

### **train record**

In [ ]:
st = time.time()
q = input("Are you sure? Y/N :")
if q == 'Y':
    !python /content/coronary-artery-stenosis-detection/generate_tfrecord.py -x /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/images/train -l /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/annotations/label_map.pbtxt -o /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/annotations/train.record
else:
    print('Moving on ...')
et = time.time()
elapsed_time = et - st
print('\nGenerated train.record successfully')
print('Execution time:', elapsed_time/60 , 'minutes')

### **test record**

In [ ]:
st = time.time()
q = input("Are you sure? Y/N :")
if q == 'Y':
    !python /content/coronary-artery-stenosis-detection/generate_tfrecord.py -x /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/images/test -l /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/annotations/label_map.pbtxt -o /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/annotations/test.record
else:
    print('Moving on ...')
et = time.time()
elapsed_time = et - st
print('\nGenerated train.record successfully')
print('Execution time:', elapsed_time/60 , 'minutes')

# **Download & extract pretrained model**

In [ ]:
%cd /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/pre-trained-models
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
!tar -xzvf /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/pre-trained-models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
!rm /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/pre-trained-models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
print("\nDownload & extraction completed")

# **Configure training pipeline**

In [ ]:
%cd /content/
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
my_model_path = trained_model_path + "my_FRCNN/"
CONFIG_PATH = my_model_path + "pipeline.config"

In [ ]:
try:
    os.mkdir(my_model_path)
except FileExistsError :
    print("my FRCNN already exists")
!cp {"/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/pre-trained-models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config"} {my_model_path}
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
CONFIG_PATH

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.faster_rcnn.num_classes = 1
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = pretrained_model_path + "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/checkpoint/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.use_bfloat16 = False
pipeline_config.train_input_reader.label_map_path= annotation_path + "label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotation_path + "train.record"]
pipeline_config.eval_input_reader[0].label_map_path = annotation_path + "label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [annotation_path + "test.record"]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

In [ ]:
updated_config_path = my_model_path + "pipeline.config"
updated_config = config_util.get_configs_from_pipeline_file(updated_config_path)
updated_config

# **Training the model**

### **Configuring GPU**

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
gpus

### **Training model**

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN --pipeline_config_path=/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN/pipeline.config --num_train_steps=20000

# **Evaluating the model**

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN --pipeline_config_path=/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN/pipeline.config --checkpoint_dir=/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN


# **Exporting model**

In [ ]:
os.mkdir("/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/exported-models/my_model_20000")

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN/pipeline.config --trained_checkpoint_dir /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/trained_model/my_FRCNN --output_directory /content/drive/MyDrive/workspace/stenosis_detection_FRCNN/exported-models/my_model_20000

# **Inferencing trained model**

In [ ]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/exported-models/my_model_20000'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/annotations/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.80)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/exported-models/my_model_20000/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

# **Detecting stenosis**

In [ ]:
# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/content/drive/MyDrive/workspace/stenosis_detection_FRCNN/images/test/14_007_10_0071.bmp'

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=1,
      min_score_thresh=0.3,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
imS = cv2.resize(image_with_detections, (720, 540))
cv2_imshow(imS)
# CLOSES WINDOW ONCE KEY IS PRESSED